In [1]:
import pandas as pd

In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [24]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%load_ext blackcellmagic

In [2]:
df_loans_backup = pd.read_csv('/home/adarsh/dattada/jupyter/loan-data/loan.csv')

/home/adarsh/anaconda3/envs/pycaret_test/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df_loans_backup.copy(deep = True)

In [4]:
import preprocess as pre

In [28]:
import EDA as eda

In [9]:
date_columns = ["issue_d","earliest_cr_line"]

In [10]:
df = pre.object_to_date(df,columns = date_columns)

In [14]:
df = df[df['application_type'] == 'Individual']

In [36]:
df = pre.drop_columns(df)

In [37]:
df = pre.emp_length_to_numeric(df)